# Bitcoin Blockchain Analysis - Getting Started

This notebook demonstrates basic queries and analysis using the Bitcoin Analysis Stack.

In [ ]:
import os
from bitcoinrpc.authproxy import AuthServiceProxy
from py2neo import Graph
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# Configuration from environment
BITCOIN_RPC = f"http://{os.getenv('BITCOIN_RPC_USER')}:{os.getenv('BITCOIN_RPC_PASSWORD')}@{os.getenv('BITCOIN_RPC_HOST')}:{os.getenv('BITCOIN_RPC_PORT')}"
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_AUTH = (os.getenv('NEO4J_USER'), os.getenv('NEO4J_PASSWORD'))
GRAPHQL_ENDPOINT = os.getenv('GRAPHQL_ENDPOINT')

## 1. Connect to Bitcoin Core RPC

In [ ]:
btc = AuthServiceProxy(BITCOIN_RPC)

# Get blockchain info
info = btc.getblockchaininfo()
print(f"Chain: {info['chain']}")
print(f"Blocks: {info['blocks']:,}")
print(f"Size on disk: {info['size_on_disk'] / 1e9:.2f} GB")
print(f"Difficulty: {info['difficulty']:,.2f}")

## 2. Query a Specific Block

In [ ]:
# Get genesis block
genesis_hash = btc.getblockhash(0)
genesis = btc.getblock(genesis_hash, 2)  # Verbosity 2 includes full tx details

print(f"Genesis Block Hash: {genesis['hash']}")
print(f"Timestamp: {genesis['time']}")
print(f"Number of transactions: {len(genesis['tx'])}")

# Show coinbase transaction
coinbase_tx = genesis['tx'][0]
print(f"\nCoinbase TX ID: {coinbase_tx['txid']}")

## 3. Connect to Neo4j Graph Database

In [ ]:
graph = Graph(NEO4J_URI, auth=NEO4J_AUTH)

# Check connection
result = graph.run("MATCH (n) RETURN count(n) as node_count").data()
print(f"Total nodes in graph: {result[0]['node_count']:,}")

## 4. Query Neo4j - Find Connected Addresses

In [ ]:
# Example: Find most active addresses
query = """
MATCH (a:Address)<-[r:OUTPUTS_TO]-(t:Transaction)
RETURN a.address as address, count(t) as tx_count, sum(r.value) as total_received
ORDER BY tx_count DESC
LIMIT 10
"""

result = graph.run(query).data()
df = pd.DataFrame(result)
print(df)

## 5. Use GraphQL API

In [ ]:
# Setup GraphQL client
transport = RequestsHTTPTransport(url=GRAPHQL_ENDPOINT)
client = Client(transport=transport, fetch_schema_from_transport=True)

# Query blockchain info
query = gql("""
    query {
        blockchainInfo {
            blocks
            chain
            difficulty
            sizeOnDisk
        }
    }
""")

result = client.execute(query)
print(result)

## 6. Visualize Transaction Graph

In [ ]:
# Example: Visualize connections for a specific address
# Replace with an actual address from your database

def visualize_address_connections(address, max_connections=20):
    query = f"""
    MATCH (a1:Address {{address: '{address}'}})<-[:OUTPUTS_TO]-(t:Transaction)-[:OUTPUTS_TO]->(a2:Address)
    WHERE a1 <> a2
    RETURN a1.address as from_addr, a2.address as to_addr, count(t) as tx_count
    LIMIT {max_connections}
    """
    
    result = graph.run(query).data()
    
    if not result:
        print(f"No connections found for address: {address}")
        return
    
    # Create NetworkX graph
    G = nx.DiGraph()
    
    for record in result:
        G.add_edge(record['from_addr'][:8], record['to_addr'][:8], weight=record['tx_count'])
    
    # Plot
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', 
            node_size=1500, font_size=8, arrows=True)
    plt.title(f"Address Connections for {address[:16]}...")
    plt.show()

# Example usage (uncomment when you have data):
# visualize_address_connections('your_address_here')

## 7. Analyze Block Statistics

In [ ]:
# Analyze last 100 blocks
current_height = btc.getblockcount()
block_stats = []

for height in range(current_height - 100, current_height):
    block_hash = btc.getblockhash(height)
    block = btc.getblock(block_hash, 1)
    
    block_stats.append({
        'height': height,
        'size': block['size'],
        'tx_count': len(block['tx']),
        'time': block['time']
    })

df_blocks = pd.DataFrame(block_stats)

# Plot
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(df_blocks['height'], df_blocks['size'])
axes[0].set_title('Block Size')
axes[0].set_xlabel('Block Height')
axes[0].set_ylabel('Size (bytes)')

axes[1].plot(df_blocks['height'], df_blocks['tx_count'], color='orange')
axes[1].set_title('Transactions per Block')
axes[1].set_xlabel('Block Height')
axes[1].set_ylabel('TX Count')

plt.tight_layout()
plt.show()

print(f"\nAverage block size: {df_blocks['size'].mean():,.0f} bytes")
print(f"Average transactions per block: {df_blocks['tx_count'].mean():.1f}")